# RESULTAT BACC TOAMASINA 2023 (Université Toamasina)

Lien:https://bacc.univ-toamasina.mg


In [ ]:
#importation des bibliothèques
import warnings
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
warnings.filterwarnings("ignore")

In [ ]:
#importation de l'url du sites
link="https://bacc.univ-toamasina.mg"
answer= requests.get(link, verify=False)
answer.status_code


In [ ]:
#création d'un session pour gérer les coockies
session = requests.Session()
answer= session.get(link, verify=False)
soup = bs(answer.content,"html.parser")
csrf = soup.find("form").find_all("input")[0]["value"]
data= {
    "q":"",
    "csrfmiddlewaretoken": csrf,
    "candidat":" ",
    "chercher":"Chercher"
}

response = session.post(link, data=data,verify=False)

#vérification
if response.status_code == 200:
    print("200")
else:
    print(response.status_code)


In [ ]:
soup1=bs(response.content,"html.parser")

In [ ]:
#scraper les données
#parcourir le tableau contenant les données
table = soup1.find("table")
cmp=0
all_data=[]

#parcourir le tableau
for information in table.find("tbody").find_all("tr"):
    #cmp=cmp+1
    #print(cmp)

#creation d'un dictionnaire dans lequel on va stocker les données concernant les étudiants
    data_student={}

#Extraction de donnée
    data=information.find_all("td")

#Ajout des éléments dans le dictionnaire vide
    cle = ['Numero','Nom','Prenom','Série','Mention','Centre','Etablissement']
    student_tem={}
    for i,donné in enumerate(data) :
        student_tem[cle[i]] = data[i].text

#Ajout du dictionnaire dans un liste créé précédemment
    all_data.append(student_tem)

In [ ]:
#Manipulation des données
df= pd.DataFrame(all_data)
df.info()
df.tail()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19879 entries, 0 to 19878
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Numero         19879 non-null  object
 1   Nom            19879 non-null  object
 2   Prenom         19879 non-null  object
 3   Série          19879 non-null  object
 4   Mention        19879 non-null  object
 5   Centre         19879 non-null  object
 6   Etablissement  19879 non-null  object
dtypes: object(7)
memory usage: 1.1+ MB


,Numero,Nom,Prenom,Série,Mention,Centre,Etablissement
19874,956 0170,SOAVELO,Marie Francelette,TTER,Passable,FENERIVE-EST,LTP Technologique Fénérive-Est
19875,956 0171,RAMANAMPISOA VOLOLONIAINA,Lovasoa Charlotte,TTER,Assez bien,FENERIVE-EST,LTP Technologique Fénérive-Est
19876,956 0174,MALINKOURATSI,Jeudinot Franki,TTER,Passable,FENERIVE-EST,LTP Technologique Fénérive-Est
19877,956 0176,IGNARAKA,José Christalino,TTER,Passable,FENERIVE-EST,LTP Technologique Fénérive-Est
19878,956 0177,IGNARAKA,Jose Grazielo,TTER,Passable,FENERIVE-EST,LTP Technologique Fénérive-Est


In [ ]:
#chargez le fichier
df.to_csv('Resultat_toamasina_2023.csv')

# DEPECHE EN FRANÇAIS (DANS ORANGE ACTU)

Lien:https://actu.orange.mg/depeches/#page-1

In [ ]:
#importation des bibliothèques
import requests
from bs4 import BeautifulSoup as bs
#import panda
import pandas as pd

In [ ]:
#Liens contenant les dépêches en Français
link_fr="https://www.orange.mg/api/depeches/langue/1/start/93433/length/20"
answer_fr=requests.get(link_fr)
answer_fr.status_code

200

In [ ]:
#Création d'un liste pour stocker les données
all_data=[]

#Extraction des données
count = requests.get("https://www.orange.mg/api/depeches/langue/1/count").json()
i=0
cmp=0

#on va boucler les liens pour obtenir tous les fichiers json dans la page
while(i<count):
    #cmp=cmp+1
    #print(cmp)
    page=requests.get(f"https://www.orange.mg/api/depeches/langue/1/start/{i}/length/300").json()
    i += 300

    #concatenation des pages
    all_data.extend(page)


In [ ]:
df=pd.DataFrame(all_data)
df.head()
#convertion de la du temps en date
def time_to_date(x):
    try :
        return pd.Timestamp(x,unit="ms")
    except:
        return None

date_publication= df['dataPublication'].apply(time_to_date)
df['time_to_date']= date_publication

In [ ]:
#chargez le fichier
df.to_csv('Orange_actu.csv')

# INFORMATION DES PUBLICATIONS(DANS MOOV)

Lien :https://moov.mg/forum/category/9

In [ ]:
#importation des bibliothèque
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [ ]:
#verification du liens
link="https://moov.mg/forum/category/9"
answers= requests.get(link)
answers.status_code

200

In [ ]:
#Extraction des données
#Création d'un liste vide pour contenir les données
all_data=[]

#obtention de tous les liens de la première pages
soup=bs(answers.content,"html.parser")
forum = soup.find_all(class_="forum-name")
#Parcourir la page

link_forum =[]
for link in forum:
    a_element = link.find("a")
    if a_element is None:
        continue

    href = "https://moov.mg" + a_element.get("href")
    link_forum.append(href)
for link in link_forum:
#entrer dans le lien pour extracter les données
#boucler la page pour avoir les pages
    next_url=link

    while True:
        resp=requests.get(next_url)
        soup = bs(resp.content, "html.parser")
        page=soup.find_all(class_="topic-name")
        url_page=[]
        for url in page:
            a= url.find("a")
            if a is  None:
                continue
            href_page="https://moov.mg"+a.get("href")
            url_page.append(href_page)

        for post_link in url_page:
            resp = requests.get(post_link)
            soup2 = bs(resp.content, "html.parser")

            info_pub={}

            data=soup2.find(class_="topic-body")
            try:
                author = data.find(class_="author-name").get_text()
            except:
                author=None

            try:
                titre=data.find(class_="topic-header").find("a").get_text()
            except:
                titre=None

            try:
                time = data.find(class_="topic-header").find("time").get_text()
            except:
                time = None
            try:
                publication=data.find(class_="post-content").find("p").get_text()
            except:
                publication = None
            info_pub.update({
                "author":author,
                "titre":titre,
                "time":time,
                "publication":publication
            })


            all_data.append(info_pub)

        next_url = soup.find(class_="pagination").find_all(class_="page-item")[-1]
        if "active" in next_url.get("class"):
            break

        next_url ="https://moov.mg"+next_url.find('a').get("href")


In [ ]:
#manipulation des données avec panda
df=pd.DataFrame(all_data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1617 entries, 0 to 1616
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   author       1616 non-null   object
 1   titre        1616 non-null   object
 2   time         1616 non-null   object
 3   publication  1609 non-null   object
dtypes: object(4)
memory usage: 50.7+ KB


In [ ]:
#chargez le fichier
df.to_csv('Moov.csv')

# FAITS DIVERS DANS MIDI MADAGASIKARA

Lien :https://midi-madagasikara.mg/category/faits-divers/

In [ ]:
#importation des bibliothèque
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [ ]:
#verification du liens
link="https://midi-madagasikara.mg/category/faits-divers/"
answers= requests.get(link)
answers.status_code

200

In [ ]:
#Extraction des données
#liste pour stocker tous les données
all_data=[]

#obtention de la première page
soup=bs(answers.content,"html.parser")

#Prendre les liens dans la page
#boucler pour obtenir tous les liens
page=[]
next_url="https://midi-madagasikara.mg/category/faits-divers/"
while True:
    resp=requests.get(next_url)
    soup=bs(resp.text,"html.parser")
    data = soup.find_all(class_="td-module-meta-info")

    for link in data:
        a=link.find("a")
    if a is None:
        continue
    href=a.get("href")
    page.append(href)
    next_url = soup.find(attrs={"aria-label":"next-page"})
    if next_url is None:
        break
    next_url =next_url.get("href")

for all_link in page:
    info_pub={}
    resp=requests.get(all_link)
    soup1=bs(resp.content,"html.parser")

    try:
        title=soup1.find(class_="tdb-title-text").get_text()
    except:
        title=None

    try:
        text=soup1.find(class_="tdi_97").find(class_="tdb-block-inner").get_text()
    except:
        text=None
    try:
        time=soup1.find("time").get_text()
    except:
        time=None
    try:
        container=soup1.find("span",class_="reading-time-number")
        reading_time=container.text+container.next_sibling.text
    except:
        reading_time=None

    info_pub.update({
                "text":text,
                "titre":title,
                "time":time,
                "reading_time":reading_time
            })
    #Ajout de tous les donnée extrait dans un liste
    all_data.append(info_pub)


In [ ]:
#Manipulation des données avec panda
df=pd.DataFrame(all_data)
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Empty DataFrame


In [ ]:
#chargez le fichier
df.to_csv('Midi_Mada.csv')

# PROPERTY AND HOUSES FOR SALE IN WELKOM (FROM REMAX)

Lien:https://www.remax.co.za/property/for-sale/south-africa/free-state/welkom/

In [ ]:
#importation de bibliothèque
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [ ]:
link="https://www.remax.co.za/property/for-sale/south-africa/free-state/welkom/"
answer= requests.get(link)
answer.status_code

200

In [ ]:
#Extraction des données
all_data=[]
soup=bs(answer.content,"html.parser")

next_url="https://www.remax.co.za/property/for-sale/south-africa/free-state/welkom/"
#Obtenir tous les liens de toutes les pages
while True :
    page=[]
    reps=requests.get(next_url)
    soup1=bs(reps.content,"html.parser")
    data = soup1.find_all(class_="outer-container")
    for link in data :
        a=link.find("a")
        if a is None:
            continue
        href="https://www.remax.co.za"+a.get("href")
        page.append(href)


#print(page)
    for link_page in page:

        reps=requests.get(link_page)
        soup2=bs(reps.content,"html.parser")
        data_page=soup2.find(attrs={"id":"listing-info"})

        details_page={}
        details=data_page.find(class_="details property-details")
        table=details.find("table")

        try:
            address=data_page.find(class_="address").get_text()
        except:
            address = None
        try:
            price =data_page.find(class_="listing_price").get_text()
        except:
            price = None

        details_page.update({
            "address":address,
            "price":price
            })

        for feature in data_page.find(class_="mobi-features-list").find_all("li"):
            cle=feature.contents[0]
            val=feature.contents[1].get_text()
            if val=="":
                val="ok"
            details_page.update({
                cle:val
            })

        for information in table.find_all("tr"):
            td=information.find_all("td")
            details_page.update({
            td[0].text:td[1].text
            })

        all_data.append(details_page)

    next_url=soup1.find(attrs={"rel":"next"})
    last_url=soup1.find_all(class_="page-item")[-1]
    if "disabled" in last_url.get("class"):
        break
    next_url=next_url.get("href")


In [ ]:
df=pd.DataFrame(all_data)
df.shape

(124, 41)

In [ ]:
#chargez le fichier
df.to_csv('Remax.csv')

# PROPERTY FROM RAWSON

Liens:https://rawson.co.za/property/for-sale/mpumalanga

In [ ]:
#importation des bibliothèque
import requests
from bs4 import BeautifulSoup as bs
#import panda
import pandas as pd
import re

In [ ]:
link="https://rawson.co.za/property/for-sale/mpumalanga"
answer= requests.get(link)
answer.status_code

200

In [ ]:
#extraction de donnée
all_data=[]
soup=bs(answer.content,"html.parser")
next_url="https://rawson.co.za/property/for-sale/mpumalanga"
page=[]
while True:
  reps=requests.get(next_url)
  soup1=bs(reps.content,"html.parser")
  data = soup1.find(class_="slice slice--1-2")
  data2 =data.find_all(class_="card")
  for link in data2:
      a=link.find("a")
      if a is None:
          continue
      href=a.get("href")
      page.append(href)

  next_url = soup1.find(attrs={"rel":"next"})
  if next_url is None:
      break
  next_url =next_url.get("href")

for link_page in page:
    info={}
    resp=requests.get(link_page)
    soup2=bs(resp.content,"html.parser")
    feature=soup2.find(class_="l-section__main").find_all(class_="features")

    try:
        price=soup2.find(class_="hero__price").text
    except:
        price=None

    for feat in feature:
        feature_label=feat.find_all(class_="features__label")
    #utilisation de regex
        for element in feature_label:
            match=re.match(r"(\w+ \w+) (\d+.+)",element.text)
            if match:
                key=match.group(1)
                values=match.group(2)
                info.update({key:values})

            match1=re.match(r"^(\d+) (\w+)",element.text)
            if match1:
                key=match1.group(2)
                values=match1.group(1)
                info.update({key:values})

    info.update({
    "price":price
})
    all_data.append(info)


In [ ]:
df=pd.DataFrame(all_data)
df.shape
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 509 entries, 0 to 508
Data columns (total 25 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ERF Size       509 non-null    object
 1   Bedrooms       426 non-null    object
 2   TV             148 non-null    object
 3   Bathrooms      322 non-null    object
 4   Garages        207 non-null    object
 5   price          509 non-null    object
 6   Building Size  107 non-null    object
 7   Lounge         332 non-null    object
 8   Bathroom       94 non-null     object
 9   Dining         247 non-null    object
 10  Garage         108 non-null    object
 11  Store          83 non-null     object
 12  Carports       109 non-null    object
 13  Family         53 non-null     object
 14  Study          77 non-null     object
 15  Fireplace      49 non-null     object
 16  Parking        111 non-null    object
 17  Carport        95 non-null     object
 18  Flatlet        67 non-null    

In [ ]:
#chargez le fichier
df.to_csv('Rawson.csv')

# PROPERTY FROM SAHOMETRADERS

Liens:https://www.sahometraders.co.za/property-for-sale-in-gauteng-p1?propertytypes=houses,apartments-flats,townhouses

In [ ]:
#import la bibliothèque request et beautifulsoup ,des outils adapté pour scraper la page web
import requests
from bs4 import BeautifulSoup as bs
#import panda
import pandas as pd

In [ ]:
#importer l'url du site à scraper dans un variable
link ="https://www.sahometraders.co.za/property-for-sale-in-gauteng-p1?propertytypes=houses,apartments-flats,townhouses"
answer=requests.get(link)
answer.status_code

200

In [ ]:
def page_web(link):
    answer= requests.get(link)
    soup = bs(answer.content,"html.parser")
    data=soup.find(class_="sc_panelWrapper sc_searchResultsP24Style")
    all_data=[]
    for info in data.find_all(class_="p24_content"):
    #creation d'un dictionnaire pour mettre les information dans chaque appartemment
        data_appartement_info={}

        try:
            price=info.find(class_="p24_price").get_text()
        except:
            price=None

        try:
            location=info.find(class_="p24_location").get_text()
        except:
            location=None

        try:
            description=info.find(class_="p24_propertyTitle").get_text()
        except:
            description=None

        bedrooms_number = info.find(class_="p24_featureDetails",title="Bedrooms")
        try:
            bedrooms_number=bedrooms_number.find("span").get_text()
        except:
            bedrooms_number = None

        bathrooms_number= info.find(class_="p24_featureDetails", title="Bathrooms")
        try:
            bathrooms_number = bathrooms_number.find("span").get_text()
        except:
            bathrooms_number = None

        parking_spaces = info.find(class_="p24_featureDetails", title="Parking Spaces")
        try :
            parking_spaces= parking_spaces.find("span").get_text()
        except:
            parking_spaces = None

        Floor_Size = info.find(class_="p24_size", title="Floor Size")
        try:
            Floor_Size = Floor_Size.find("span").get_text()
        except:
            Floor_Size = None

        erf_size = info.find(class_="p24_size", title="Erf Size")
        try:
            erf_size =erf_size .find("span").get_text()
        except:
            erf_size = None
        #Ajout des éléments dans un dictionnaire
        data_appartement_info=({
            "Price":price,
            "Location":location,
            "Description":description,
            "Bedroom":bedrooms_number,
            "Bathroom":bathrooms_number,
            "Parking_space":parking_spaces,
            "Floor_size":Floor_Size,
            "Erf_size":erf_size,
            "link":link
        })

        all_data.append(data_appartement_info)
        #print(all_data)
    next_url=soup.find("div",class_="text-center").find("span",class_="pull-right text-muted").find("a").get("href")
    return all_data, next_url

In [ ]:
all_page =[]
url = link
page_num = 0
while url != "":
  page_num += 1
  print(page_num)
  ds, url = page_web(url)
    #Concatenation des listes
  all_page.extend(ds)

In [ ]:
df=pd.DataFrame(all_page)
df["Price"]=df["Price"].str.strip()
def type(x):
    if x is None:
        return None
    elif x.find("House") != -1:
        return "House"
    elif x.find("Townhouse") != -1:
        return "Townhouse"
    elif x.find("Apartement") !=-1 or x.find("Flat") != -1:
        return "Apartment/Flat"

df["Property"]= df["Description"].apply(type)
df.head()
df.shape

(28959, 10)

In [ ]:
#chargez le fichier
df.to_csv('Sahometraders.csv')

# TROVIT

In [ ]:
#importation des bibliothèque nécéssaire
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [ ]:
link="https://homes.trovit.co.za/index.php/cod.search_homes/type.1/what_d.eastern%20cape/isUserSearch.1"
answer=requests.get(link)
answer.status_code

200

In [ ]:
all_data=[]

soup= bs(answer.content,"html.parser")
data=soup.find(attrs={"id":"content_center"})
next_url="https://homes.trovit.co.za/index.php/cod.search_homes/type.1/what_d.eastern%20cape/isUserSearch.1"

page_num = 0
while True:
    page_num += 1
    print(page_num)
    reps=requests.get(next_url)
    soup1=bs(reps.content,"html.parser")
    data=soup1.find(attrs={"id":"content_center"})
    for info in data.find_all(class_="snippet-content-main"):
            data_appartement_info={}
            try:
                price=info.find(class_="item-price").get_text()
            except:
                price=None

            try:
                location=info.find(class_="item-address").get_text()
            except:
                location=None

            try:
                title=info.find(class_="item-title").get_text()
            except:
                title=None

            bedrooms_number = info.find(class_="item-property item-rooms")
            try:
                bedrooms_number=bedrooms_number.find("span").get_text()
            except:
                bedrooms_number = None

            bathrooms_number= info.find(class_="item-property item-baths")
            try:
                bathrooms_number = bathrooms_number.find("span").get_text()
            except:
                bathrooms_number = None

            Size = info.find(class_="item-property item-size")
            try:
                Size = Size.find("span").get_text()
            except:
                Size = None

            #Ajout des éléments dans un dictionnaire
            data_appartement_info=({
                "Price":price,
                "Location":location,
                "Description":title,
                "Bedroom":bedrooms_number,
                "Bathroom":bathrooms_number,
                "Size":Size,
            })
            all_data.append(data_appartement_info)


    next_url=soup1.find("p",class_="trovit-paginator").find("a",attrs={"data-test":"p-next"})
    if next_url is None:
        break
    next_url =next_url.get("href")

In [ ]:
df=pd.DataFrame(all_data)
def type(x):
    if x is None:
        return None
    elif x.find("House") != -1:
        return "House"
    elif x.find("Townhouse") != -1:
        return "Townhouse"
    elif x.find("Apartement") !=-1 or x.find("Flat") != -1:
        return "Apartment/Flat"

df["Property"]= df["Description"].apply(type)
df.head()


,Price,Location,Description,Bedroom,Bathroom,Size,Property
0,"ZAR1,350,000",Eastern Cape,3 Bedroom House Eastern Cape For Sale At 1350-,3 bedrooms,1 bathrooms,110 m²,House
1,"ZAR5,950,000",Eastern Cape,5 Bedroom House Eastern Cape For Sale At 5950-,5 bedrooms,4 bathrooms,450 m²,House
2,"ZAR6,700,000","Cape St. Francis, Sarah Baartman District Mu...",5 Bedroom House Cape St Francis Eastern Cape ...,5 bedrooms,4 bathrooms,325 m²,House
3,"ZAR360,000",Eastern Cape,Eastern Cape N/A,-,-,-,None
4,"ZAR330,000",Eastern Cape,Eastern Cape N/A,-,-,-,None


In [ ]:
#chargez le fichier
df.to_csv('Trovit.csv')

# PROPERTY FROM CAPE TOWN

Lien: https://www.privateproperty.co.za/for-sale/western-cape/cape-town/55

In [ ]:
#import la bibliothèque request et beautifulsoup ,des outils adapté pour scraper la page web
import requests
from bs4 import BeautifulSoup as bs
#import panda
import pandas as pd
import re

In [ ]:
link="https://www.privateproperty.co.za/for-sale/western-cape/cape-town/55"
answer= requests.get(link)
answer.status_code

In [ ]:
all_link =[]
all_data=[]
soup=bs(answer.content,"html.parser")
data=soup.find(class_="linksHolder col")
#tous les liens de la première page
for link in data.find_all("li"):
    a= link.find("a")
    if a is None:
        continue
    href="https://www.privateproperty.co.za"+a.get("href")
    all_link.append(href)

for link in all_link:
    next_url=link
    while True:
        reps=requests.get(next_url)
        soup1=bs(reps.content,"html.parser")
        page=soup1.find(class_="resultsItemsContainer")
        a =page.find_all(class_="listingResult row")

        url_page=[]
        for link in a:
            href_page="https://www.privateproperty.co.za"+link.get("href")
            url_page.append(href_page)


        for post_link in url_page:
            resp1 = requests.get(post_link)
            soup2 = bs(resp1.content, "html.parser")

            info_pub={}

            all_area =soup2.find_all(class_="mainFeature")
            for element in all_area:
                key=element.find("strong").text.strip()
                values=element.find("strong").next_sibling.text.strip()
                info_pub.update({
                        key:values,
                    })

            feature =soup2.find(class_="featureCols")
            for attribut in feature.find_all(class_="attribute"):
                key=attribut.find(class_="propAttrValue").get_text()
                values=attribut.find(class_="attributeLabel").get_text()
                info_pub.update({
                        key:values,
                    })

            data=soup2.find(class_="titleContainer")
            try:
                title = data.find("h1").get_text()
            except:
                title=None

            try:
                price=data.find(class_="priceInfo").find(class_="detailsPrice").get_text()
            except:
                price=None

            try:
                address = data.find(class_="address").get_text()
            except:
                address = None
            info_pub.update({
                        "title ":title ,
                        "price[R]":price,
                        "address":address
                    })
            all_data.append(info_pub)

        next_url=soup1.find(class_="pagination").find_all(class_="pageNumber")[-1]
        if "disabled" in next_url.get("class"):
            break
        next_url="https://www.privateproperty.co.za"+next_url.get("href")
#print(all_link)


In [ ]:
df=pd.DataFrame(all_data)
df.shape



In [ ]:
#chargez le fichier
df.to_csv('Cape_town.csv')


# PROPERTY FROM PAMGOLDING

Liens:https://www.pamgolding.co.za/property-search/properties-for-sale-limpopo-province/1874

In [ ]:
#importation la bibliothèque
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import math

In [ ]:
link="https://www.pamgolding.co.za/property-search/properties-for-sale-limpopo-province/1874"
answer= requests.get(link)
answer.status_code

200

In [ ]:
soup=bs(answer.content,"html.parser")
nbr_page = math.ceil(int(soup.find("title").text.strip().split()[0])/10)
all_data=[]
next_url = link
page = 1
while page <= nbr_page:

    resp = requests.get(next_url)
    soup1=bs(resp.content, "html.parser")
    data=soup1.find_all(class_="pgp-property__item")
    #Exctraction
    for info in data:
        try:
            description=info.find(class_="pgp-description").text
        except:
            description=None
        try:
            price=info.find(class_="pgp-price").text
        except:
            price=None
        try:
            bedroom=info.find(class_="icon bedroom").find_previous_sibling(class_="text").text
        except:
            bedroom=None
        try:
            bathroom=info.find(class_="icon bathroom").find_previous_sibling(class_="text").text
        except:
            bathroom=None
        try:
            erf_size=info.find(class_="icon erf-size").find_next_sibling(class_="text").text
        except:
            erf_size=None
        try:
            building=info.find(class_="building-size").find_next_sibling(class_="text").text
        except:
            building=None

        try:
            parking=info.find(class_="parking").find_previous_sibling(class_="text").text
        except:
            parking=None

        try:
            garage=info.find(class_="garage").find_previous_sibling(class_="text").text
        except:
            garage=None

        data_info_appartements = {
            "price":price,
            "description": description,
            "bedroom":bedroom,
            "bathroom":bathroom,
            "erf_size":erf_size,
            "building_size": building,
            "parking":parking,
            "garage": garage
        }
        all_data.append(data_info_appartements)


    page += 1
    next_url = f"https://www.pamgolding.co.za/property-search/properties-for-sale-limpopo-province/1874/page{page}"


In [ ]:
df=pd.DataFrame(all_data)
df["price"] = df["price"].apply(lambda x: None if x=="" else x)
df.dropna(subset=["price"]).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 897 entries, 0 to 985
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   price          897 non-null    object
 1   description    897 non-null    object
 2   bedroom        473 non-null    object
 3   bathroom       468 non-null    object
 4   erf_size       859 non-null    object
 5   building_size  112 non-null    object
 6   parking        78 non-null     object
 7   garage         257 non-null    object
dtypes: object(8)
memory usage: 63.1+ KB


In [ ]:
#chargez le fichier
df.to_csv('pamgolding.csv')